In [1]:
# import timm
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn import functional as F
from typing import Optional, Dict, Tuple
from ptflops import get_model_complexity_info
from pprint import pprint
import torch
import pdb
from torch import nn
from thop import profile
import torchsummary
from utils import cfg_m0, cfg_m2
from backbone import *
%load_ext autoreload
%autoreload 2
model_name = 'micronet_m2'
model = MicroNet(cfg_m2)

[2, 0, 1]
inp: 32, oup:12, g:4
reduction: 4, squeeze: 12/4
init-a: [1.0, 0.0], init-b: [0.0, 0.0]
group shuffle: 4, divide group: False
[2, 2, 1]
reduction: 8, squeeze: 48/8
init-a: [1.0, 1.0], init-b: [0.0, 0.0]
group shuffle: 12, divide group: False
inp: 48, oup:16, g:4
reduction: 4, squeeze: 16/4
init-a: [1.0, 0.0], init-b: [0.0, 0.0]
group shuffle: 4, divide group: False
[2, 2, 1]
reduction: 8, squeeze: 64/8
init-a: [1.0, 1.0], init-b: [0.0, 0.0]
group shuffle: 16, divide group: False
inp: 64, oup:24, g:4
reduction: 4, squeeze: 24/8
init-a: [1.0, 0.0], init-b: [0.0, 0.0]
group shuffle: 4, divide group: False
[2, 2, 1]
inp: 24, oup:144, g:6
reduction: 8, squeeze: 144/20
init-a: [1.0, 1.0], init-b: [0.0, 0.0]
group shuffle: 6, divide group: False
reduction: 8, squeeze: 144/20
init-a: [1.0, 1.0], init-b: [0.0, 0.0]
group shuffle: 24, divide group: True
inp: 144, oup:32, g:4
reduction: 4, squeeze: 32/8
init-a: [1.0, 0.0], init-b: [0.0, 0.0]
group shuffle: 4, divide group: False
[2, 2, 

In [2]:
input = torch.randn(1, 3, 224, 224)
macs, params, layer_info = profile(model, inputs=(input, ), ret_layer_info=True)
print(model_name, macs/1e6)
# mobile_former_294m 293.074048

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU6'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
micronet_m2 15.06966


In [3]:
with torch.cuda.device(0):
  # net = model
  net = model
  macs, params = get_model_complexity_info(net, (3, 224, 224), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', params))

MicroNet(
  2.38 M, 99.392% Params, 12.87 MMac, 100.000% MACs, 
  (features): Sequential(
    568.35 k, 23.687% Params, 10.98 MMac, 85.281% MACs, 
    (0): StemLayer(
      84, 0.004% Params, 1.71 MMac, 13.254% MACs, 
      (stem): Sequential(
        84, 0.004% Params, 1.71 MMac, 13.254% MACs, 
        (0): SpatialSepConvSF(
          84, 0.004% Params, 1.61 MMac, 12.474% MACs, 
          (conv): Sequential(
            84, 0.004% Params, 1.61 MMac, 12.474% MACs, 
            (0): Conv2d(36, 0.002% Params, 903.17 KMac, 7.017% MACs, 3, 4, kernel_size=(3, 1), stride=(2, 1), padding=(1, 0), bias=False)
            (1): BatchNorm2d(8, 0.000% Params, 200.7 KMac, 1.559% MACs, 4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): Conv2d(24, 0.001% Params, 301.06 KMac, 2.339% MACs, 4, 8, kernel_size=(1, 3), stride=(1, 2), padding=(0, 1), groups=4, bias=False)
            (3): BatchNorm2d(16, 0.001% Params, 200.7 KMac, 1.559% MACs, 8, eps=1e-05, momentum=0.1, affi